In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import pandas as pd
import numpy as np

df = pd.read_pickle('../data/topic.pkl')

In [2]:
tokenizer = AutoTokenizer.from_pretrained("mdraw/german-news-sentiment-bert")
model = AutoModelForSequenceClassification.from_pretrained("mdraw/german-news-sentiment-bert")
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
classifier("Dies ist ein schlechter Test.")

[{'label': 'negative', 'score': 0.988132655620575}]

In [3]:
df['contains_cdu'] = df['preprocessed'].str.contains('cdu')
df['contains_csu'] = df['preprocessed'].str.contains('csu')
df['contains_fdp'] = df['preprocessed'].str.contains('fdp')
df['contains_grüne'] = df['preprocessed'].str.contains('grüne') 
df['contains_linke'] = df['preprocessed'].str.contains('linke')
df['contains_afd'] = df['preprocessed'].str.contains('afd') 
df['contains_afg'] = df['preprocessed'].str.contains('afg')
df['contains_spd'] = df['preprocessed'].str.contains('spd')

In [4]:
ind = 4
subset = 'contains_grüne'

teststring = df.loc[df[subset]].iloc[ind]['transcript']
print(f'String: {teststring}')
print(f'Classification: {classifier(teststring)}')

String: bewegungen durch kämpfe auf den straßen im rahmen der proteste dass wir es sind die die forderung stellen und auch fordern müssen wir müssen die fragen richtig stellen und wir müssen auch die richtigen antworten geben darauf danket über meine nächste frage geht ein juhu kurz vor der bundestagswahl gab es ja noch mal einen großen streit tag bei future nun sind die grünen in der bundesregierung wenn es nach der ampel geht soll die klimakatastrophe gelöst werden durch innovationen in industrie technologie und wirtschaft unterstützt durch staatliche unterstützung unterstützungsprogramme es wird immer mehr produziert ohne dabei auf die bedürfnisse der bevölkerung einzugehen so sollen zb bis 20.30 mindestens 15 millionen e-autos auf deutschen straßen rollen ein radikales umdenken in sachen klimaschutz blieb wie zu erwarten war aus viele jugendliche sehen in den grünen jedoch ein fortschritt wie geht 
Classification: [{'label': 'neutral', 'score': 0.5348684191703796}]


In [20]:
def extract_party_mentions(input_string, party_strings, n_words=10):
    party_boolean = [False for i in input_string.split()]
    for p in party_strings:
        temp = [True if string == p else False for string in input_string.split()]
        party_boolean = [party_boolean|temp for (party_boolean,temp) in zip(party_boolean, temp)]
    party_index = np.where(party_boolean)[0]
    return [" ".join(input_string.split()[party_index[i]-n_words:party_index[i]+(n_words+1)]) for i in range(len(party_index))]

In [21]:
extract_party_mentions(input_string=teststring, party_strings=['grüne', 'grünen'])

['mal einen großen streit tag bei future nun sind die grünen in der bundesregierung wenn es nach der ampel geht soll',
 'wie zu erwarten war aus viele jugendliche sehen in den grünen jedoch ein fortschritt wie geht']

In [36]:
res_series = df['transcript'].loc[df['contains_fdp']].apply(lambda transcript: extract_party_mentions(input_string=transcript, party_strings=['fdp'], n_words=10))
res_df = pd.DataFrame(res_series).explode(column='transcript')
res_df.reset_index(inplace=True)
res_df.dropna(inplace=True)

In [37]:
res_df['sentiment'] = res_df['transcript'].apply(classifier)

In [60]:
sentiment_index = 0
print(f"Phrase: {res_df['transcript'].iloc[sentiment_index]}")
print(f"Sentiment: {res_df['sentiment'].iloc[sentiment_index][0]['label']}")
print(f"Score: {res_df['sentiment'].iloc[sentiment_index][0]['score']}")

Phrase: der wahl hat die neue regierung aus spd grüne und fdp den koalitionsvertrag vorgestellt das bündnis für freiheit gerechtigkeit und
Sentiment: positive
Score: 0.9217967987060547
